# MFM 702 - Final Project: Portfolio Risk management

### Mohammad Mohammadi 

# Section1: Loading CSV files, plotting, and  processing data

In [0]:
# connecting colab to local google drive to be able to read csv files
from google.colab import drive
drive.mount("/content/drive")
base_path = "/content/drive/My Drive/Private/McMaster/Courses/David's Course/Project/Final Report"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# import libraries
import os
import numpy as np
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

## Loading CSV file data

In [0]:
# Importing csv data for TSX and BCE
TSX_data_All = pd.read_csv(f"{base_path}/TSX.csv")
TSX_data_All = TSX_data_All.iloc[::-1]
TSX_data_All.rename(columns={"Last Price":"Adj Close"}, inplace=True)
BCE_data_All = pd.read_csv(f"{base_path}/BCE.csv")

# Some modifications on Data formats
TSX_data_All["Date"] = pd.to_datetime(TSX_data_All["Date"])
BCE_data_All["Date"] = pd.to_datetime(BCE_data_All["Date"])
TSX_data_All.set_index(["Date"], inplace=True)
BCE_data_All.set_index("Date", inplace=True)

# BCE Stock data over 5 years: 
BCE_data_All.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-12-15,51.500000,52.180000,51.220001,51.759998,40.389706,1803700
2014-12-16,51.369999,52.730000,51.360001,52.349998,40.850105,2005200
2014-12-17,52.400002,52.700001,51.820000,52.610001,41.052979,1513500
2014-12-18,52.970001,53.090000,52.189999,52.970001,41.333912,1944200
2014-12-19,53.500000,53.500000,52.759998,53.020000,41.372913,5835500


## Selecting only a subset of data

In [0]:
# Only subsection of data 
start_date = "2018-11-01"
end_date = "2019-11-01"
BCE = BCE_data_All.loc[start_date:end_date, :]
BCE = BCE[["Adj Close"]]
TSX = TSX_data_All.loc[start_date:end_date, :]
# BCE Adj close data
BCE.head()

,Adj Close
Date,
2018-11-01,50.127747
2018-11-02,49.602654
2018-11-05,49.837070
2018-11-06,49.621407
2018-11-07,50.249645


## Question 4:  Plotting Prices and also Returns

In [0]:
# TSX and BCE price plotting
from plotly.subplots import make_subplots
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=TSX.index, y=TSX["Adj Close"], mode='lines', name="TSX Price"), secondary_y=False)
fig.add_trace(go.Scatter(x=BCE.index, y=BCE["Adj Close"], mode='lines', name="BCE Price"), secondary_y=True)
fig.update_yaxes(title_text="TSX Price", secondary_y=False)
fig.update_yaxes(title_text="BCE Pric", secondary_y=True)
fig.update_yaxes(automargin=True)

fig.show()

In [0]:
# log returns of TSX and BCE
TSX["Log Returns"] = np.log(TSX["Adj Close"]/TSX['Adj Close'].shift(1))
BCE["Log Returns"] = np.log(BCE["Adj Close"]/BCE['Adj Close'].shift(1))

In [0]:
# Plotting Log returns 
fig = go.Figure()
fig.add_trace(go.Scatter(x=TSX.index, y=TSX["Log Returns"], mode='lines', name="TSX log returns"))
fig.add_trace(go.Scatter(x=BCE.index, y=BCE["Log Returns"], mode='lines', name="BCE log returns"))

### Drift, volatility, and correlation calculations

In [0]:
BCE_drift = np.mean(BCE["Log Returns"])*252
BCE_volatility = np.std(BCE["Log Returns"]) * np.sqrt(252)
print(f"BCE drift: {BCE_drift*100:.2f}%,  BCE volatility: {BCE_volatility*100:.2f}%")

TSX_drift = np.mean(TSX["Log Returns"])*252
TSX_volatility = np.std(TSX["Log Returns"]) * np.sqrt(252)
print(f"TSX drift: {TSX_drift*100:.2f}%,  TSX volatility: {TSX_volatility*100:.2f}%")

BCE_TSX_correlation = BCE["Log Returns"].corr(TSX["Log Returns"])
print(f"Correlation between BCE Inc. and TSX is: {BCE_TSX_correlation*100:0.2f}%")

BCE drift: 20.82%,  BCE volatility: 10.49%
TSX drift: 9.72%,  TSX volatility: 9.64%
Correlation between BCE Inc. and TSX is: 31.58%


# Section 2: Value at Risk (VAR) and Conditional Value at Risk (CVAR) calculations 

## Question 5: Theoretical VAR and CVAR calculations

Here the following formula can be used to calculate VAR for portfolio: 

$ VAR_{\Delta t} = N^{-1}(99\%) \sigma_\pi \sqrt{\Delta t} $

In [0]:
from scipy.stats import norm

delta_TSX = 0.0340288
delta_BCE = 0.313096
S_TSX  = 989.12
S_BCE  = 62.78 
realized_Loss = 35906.62
sigma_TSX  = delta_TSX * S_TSX * TSX_volatility/ np.sqrt(52)  # in $ value over a week 
sigma_BCE = delta_BCE * S_BCE * BCE_volatility / np.sqrt(52) # in $ value over a week 

sigma_portfolio = np.sqrt(sigma_TSX**2 + sigma_BCE**2 + 2* BCE_TSX_correlation * sigma_TSX * sigma_BCE )
print(f"Portfolio volatility : ${sigma_portfolio:.2f}")

# Theoretical VAR for 95th and 99th percentile
Var_95 = norm.ppf(0.95)* sigma_portfolio
Var_99 = norm.ppf(0.99)* sigma_portfolio
print(f"95th Percentile VAR: ${Var_95:.2f}          99th Percentile VAR: ${Var_99:.2f}")
print(f"Movement in value in Nov. 6 was: {realized_Loss/sigma_portfolio:.0f} times of portfolio standard deviation")

Portfolio volatility : $0.60
95th Percentile VAR: $0.99          99th Percentile VAR: $1.41
Movement in value in Nov. 6 was: 59404 times of portfolio standard deviation


## Question 6: Portfolio Efficient Frontier

In [0]:
n = 1000
r=0.02
w_BCE = np.arange(0,100,0.1)
w_TSX = 100-w_BCE
mu_portfolio = w_BCE * BCE_drift + w_TSX * TSX_drift
sigma_squared_portfolio = BCE_volatility**2 * w_BCE**2 + TSX_volatility**2 * w_TSX**2 + 2* BCE_TSX_correlation * BCE_volatility*TSX_volatility*w_BCE*w_TSX
sigma_portfolio = np.sqrt(sigma_squared_portfolio)
sharp_ratio = (mu_portfolio - r)/sigma_portfolio
sharp_ratio


layout = go.Layout(xaxis=dict(range=[7, 12]), yaxis=dict(range=[5, 25]))
fig = go.Figure(layout=layout)
fig.add_trace(go.Scatter(x=sigma_portfolio, y=mu_portfolio))
fig.update_layout(xaxis_title="Portfolio volatility($)", yaxis_title="Portfolio return($)")

fig.show()


## Question 7: Asian Option Pricing using Interpolation

In [0]:
Asian_option_prices = pd.read_csv(f"{base_path}/Asian_option_prices.csv", header=None)
Asian_option_prices.columns = ["Stock Price", "Option Price"]
Asian_option_prices.tail()

,Stock Price,Option Price
25,1004,27.643950
26,1006,29.344880
27,1010,32.757404
28,1015,37.033066
29,1020,41.312839


Function that uses the excel data to calculate option prices using linear interpolation

In [0]:
from scipy import interpolate
stock_price = Asian_option_prices["Stock Price"] 
Option_price = Asian_option_prices["Option Price"]

# interpolation function for interpolating Asian call
Asian_Call_Pricing_Interpolation = interpolate.interp1d(stock_price, Option_price, fill_value="extrapolate")

In [0]:
new_Stock_Price = [940, 960, 967, 970, 973,975, 1100]
Asian_Call_Pricing_Interpolation(new_Stock_Price)

array([2.89933600e-02, 1.16500000e+00, 2.86293340e+00, 3.83200000e+00,
       5.17882120e+00, 6.07670200e+00, 1.09789209e+02])

# Question 8: Mont Carlo Simulation


In [0]:
# calculate call option price
def call_option(t, S, K, r, sigma, T):
  tau = (T-t).days/365
  d2 = (np.log(S/K) + (r- sigma**2/2)*tau)/(sigma * np.sqrt(tau))
  d1 = d2 + sigma*np.sqrt(tau)
  call = S * norm.cdf(d1) - np.exp(-r*tau)* K * norm.cdf(d2)
  return call

# calculate Put option price
def put_option(t, S, K, r, sigma, T):
  tau = (T-t).days/365
  d2 = (np.log(S/K) + (r- sigma**2/2)*tau)/(sigma * np.sqrt(tau))
  d1 = d2 + sigma*np.sqrt(tau)
  put = np.exp(-r*tau)* K * norm.cdf(-d2) - S * norm.cdf(-d1)
  return put

# calculate Put option price
def digital_put_option(t, S, K, r, sigma, T):
  tau = (T-t).days/365
  d2 = (np.log(S/K) + (r- sigma**2/2)*tau)/(sigma * np.sqrt(tau))
  d1 = d2 + sigma*np.sqrt(tau)
  digital_put = np.exp(-r*tau)* norm.cdf(-d2)
  return digital_put


In [0]:
# Test option funcions 
call_option(S=989.12, K=1000, r=0.02, sigma=0.082, T=datetime(2019,12,20),t=datetime(2019,10,31)) # should be 8.39
# digital_put_option(S=64.1, K=60.3, r=0.02, sigma=0.184, T="2019-12-11", t="2019-11-14") # 0.11
# put_option(S=64.1, K=56, r=0.02, sigma=0.21, T="2020-01-17", t="2019-11-14") # 0.13

8.38902877576652

Using CAPM for simulation: 
$\epsilon_{BCE} = \rho * \epsilon_{TSX} + \sqrt{1-\rho^2} \epsilon_1 $ 

In [0]:
# Mont-Carlo simulation for Stock prices
def mont_carlo(n_samples):
  # parameters
  delta_t = 7/365.
  S0_BCE = 62.78
  S0_TSX = 989.12

  rho = BCE_TSX_correlation
  eps_TSX = norm.rvs(size=n_samples)
  eps_1 = norm.rvs(size=n_samples)
  eps_BCE = rho *eps_TSX + np.sqrt(1-rho**2)*eps_1

  S_TSX = S0_TSX* np.exp((TSX_drift - TSX_volatility**2/2 )*delta_t + TSX_volatility*np.sqrt(delta_t)*eps_TSX)
  S_BCE = S0_BCE* np.exp((BCE_drift - BCE_volatility**2/2 )*delta_t + BCE_volatility*np.sqrt(delta_t)*eps_BCE)

  return S_TSX, S_BCE

In [0]:
# calculating portfolio value for TSX and BCE prices and any time t
def calculate_portfolio_value(t, S_TSX, S_BCE):
  # TSX positions 
  Asian_call_Option = Asian_Call_Pricing_Interpolation(S_TSX)
  TSX_call_option  = call_option(t=t, S=S_TSX, K=1000,r=0.02, sigma= 0.082, T=datetime(2019,12,20) )

  # BCE Stock positions
  BCE_digital_put = digital_put_option(t=t, S=S_BCE, K=60.3, r=0.02, sigma=0.18, T=datetime(2019,12,11) )
  BCE_Put = put_option(t=t, S=S_BCE, K=60, r=0.02, sigma=0.2, T=datetime(2019,12,20))

  # Portfolio value 
  TSX_positions = -513 * S_TSX - 20000 * Asian_call_Option + 35000 * TSX_call_option
  BCE_positions = -7038 * S_BCE - 233384 * BCE_digital_put + 50000 * BCE_Put
  portfolio_value = TSX_positions + BCE_positions
  return portfolio_value, TSX_positions,BCE_positions, Asian_call_Option, TSX_call_option, BCE_digital_put, BCE_Put

In [0]:
# Mont-Carlo simulation for generating TSX and BCE prices 
n_samples = 10000
Portfolio_0 = -1019119.33
S_TSX, S_BCE = mont_carlo(n_samples)
t= datetime(2019,11,7)
portfolio_value, TSX_positions,BCE_positions, Asian_call_Option, TSX_call_option, BCE_digital_put, BCE_Put = calculate_portfolio_value(t, S_TSX, S_BCE)
loss = Portfolio_0 - portfolio_value

loss_95_percentile = np.percentile(loss,95)
loss_99_percentile = np.percentile(loss,99)
cvar_90_percentile =  loss[loss>np.percentile(loss, 90)].mean()

print(f"Standard deviation of Portfolio changes: ${np.std(portfolio_value)}")
print(f" 95% of Portfolio losses is: ${loss_95_percentile:.2f}, which is {loss_95_percentile/np.std(portfolio_value):.2f} times of Standard deviation of portfolio changes")
print(f" 99% of Portfolio losses is: ${loss_99_percentile:.2f}, which is {loss_99_percentile/np.std(portfolio_value):.2f} times of Standard deviation of portfolio changes")
print(f" CVAR of 90% loss is: ${cvar_90_percentile:.2f}")

Standard deviation of Portfolio changes: $33015.079142991664
 95% of Portfolio losses is: $11511.22, which is 0.35 times of Standard deviation of portfolio changes
 99% of Portfolio losses is: $13403.95, which is 0.41 times of Standard deviation of portfolio changes
 CVAR of 90% loss is: $11825.22


In [0]:
# Calculation of realized_Loss percentile
from  scipy import stats
realized_Loss = 35906.62
realized_loss_percentile = stats.percentileofscore(loss, realized_Loss, 'rank')
print(f"Maximum loss from Mont-Carlo simulation is: ${max(loss):.2f}")
print(f"Realized loss is in {realized_loss_percentile} percentile of Mont-Carlo simulated loss ")

Maximum loss from Mont-Carlo simulation is: $16497.50
Realized loss is in 100.0 percentile of Mont-Carlo simulated loss 


In [0]:

fig = make_subplots(rows=3, cols=2, subplot_titles=("TSX Price", "BCE Price", "TSX Asian Call", "TSX Call", "BCE digital Put", "BCE Put"))
fig.add_trace(go.Histogram(x=S_TSX, histnorm="probability", nbinsx=100), row=1,col=1)
fig.add_trace(go.Histogram(x=S_BCE, histnorm="probability", nbinsx=100), row=1,col=2)
fig.add_trace(go.Histogram(x=Asian_call_Option, histnorm="probability", nbinsx=100), row=2,col=1)
fig.add_trace(go.Histogram(x=TSX_call_option, histnorm="probability", nbinsx=100), row=2,col=2)
fig.add_trace(go.Histogram(x=BCE_digital_put, histnorm="probability", nbinsx=100), row=3,col=1)
fig.add_trace(go.Histogram(x=BCE_Put, histnorm="probability", nbinsx=100), row=3,col=2)

fig.update_layout(showlegend=False, title_text="Mont Carlo simulation")

In [0]:
fig = make_subplots(rows=2, cols=2, subplot_titles=("TSX Position", "BCE Position", "Portfolio value", "Loss"))
fig.add_trace(go.Histogram(x=TSX_positions, histnorm="probability", nbinsx=100), row=1,col=1)
fig.add_trace(go.Histogram(x=BCE_positions, histnorm="probability", nbinsx=100), row=1,col=2)
fig.add_trace(go.Histogram(x=portfolio_value, histnorm="probability", nbinsx=100), row=2,col=1)
fig.add_trace(go.Histogram(x=loss, histnorm="probability", nbinsx=100), row=2,col=2)
fig.update_layout(showlegend=False)

In [0]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=loss,histnorm="probability", nbinsx=100, name="Loss", xbins=dict( # bins used for histogram
        start=-150000,
        end=50000,
        size=1000
    ),))
fig.add_trace(go.Scatter(x=[loss_95_percentile,loss_95_percentile], y=[0.00005,0.07], name="95% Loss"))
fig.add_trace(go.Scatter(x=[loss_99_percentile,loss_99_percentile], y=[0.00005,0.07], name="99% Loss"))
fig.update_layout(title_text="Portfolio Loss")

# Question 9 : Historical Simulation 

In [0]:
num_samples = 2000 * 5
Hist_resampled = pd.DataFrame(index=TSX.index)
Hist_resampled["TSX_Log_Return"] = TSX.loc[Hist_resampled.index,"Log Returns"]
Hist_resampled["BCE_Log_Return"] = BCE.loc[Hist_resampled.index,"Log Returns"]

# resampling with replacement
Hist_resampled = Hist_resampled.sample(n=num_samples, replace=True, )
Hist_resampled.reset_index(inplace=True)
Hist_resampled.head()

,Date,TSX_Log_Return,BCE_Log_Return
0,2019-02-07,-0.000486,-0.001217
1,2018-11-20,-0.013786,-0.006304
2,2019-03-06,0.001474,0.009382
3,2018-11-05,0.005389,0.004715
4,2019-09-04,0.002163,0.002693


In [0]:
# Calculation of Price values in 5 days
S_TSX_Oct30 = 989.12
S_BCE_Oct30 = 62.78
t= datetime(2019,11,7)
price_Nov7 = pd.DataFrame()
price_Nov7["TSX Price"] = [S_TSX_Oct30 * np.exp(Hist_resampled.iloc[i:i+5]["TSX_Log_Return"].sum()) for i in np.arange(0,num_samples,5)]
price_Nov7["BCE Price"] = [S_BCE_Oct30 * np.exp(Hist_resampled.iloc[i:i+5]["BCE_Log_Return"].sum()) for i in np.arange(0,num_samples,5)]
price_Nov7.head()

,TSX Price,BCE Price
0,983.944383,63.364607
1,989.157730,64.045275
2,1001.982794,64.872535
3,999.290970,62.195657
4,961.095939,62.449194


In [0]:
S_TSX_Bootstrapping = price_Nov7["TSX Price"]
S_BCE_Bootstrapping  = price_Nov7["BCE Price"]
portfolio_value_Bootstrapping, TSX_positions_Bootstrapping,BCE_positions_Bootstrapping, Asian_call_Option_Bootstrapping, TSX_call_option_Bootstrapping, BCE_digital_put_Bootstrapping, BCE_Put_Bootstrapping = calculate_portfolio_value(t, S_TSX_Bootstrapping, S_BCE_Bootstrapping)
loss_Bootstrapping = Portfolio_0 - portfolio_value_Bootstrapping

loss_95_percentile_Bootstrapping = np.percentile(loss_Bootstrapping,95)
loss_99_percentile_Bootstrapping = np.percentile(loss_Bootstrapping,99)
cvar_90_percentile =  loss[loss>np.percentile(loss_Bootstrapping, 90)].mean()


print(f"Standard deviation of Portfolio changes: ${np.std(portfolio_value_Bootstrapping)}")
print(f" 95% of Portfolio losses is: ${loss_95_percentile_Bootstrapping}")
print(f" 99% of Portfolio losses is: ${loss_99_percentile_Bootstrapping}") 
print(f" CVAR of 90% loss is: ${cvar_90_percentile}")

Standard deviation of Portfolio changes: $35907.64904650933
 95% of Portfolio losses is: $11411.507566143986
 99% of Portfolio losses is: $12984.34132605246
 CVAR of 90% loss is: $11965.354169471442


In [0]:

fig = make_subplots(rows=3, cols=2, subplot_titles=("TSX Price", "BCE Price", "TSX Asian Call", "TSX Call", "BCE digital Put", "BCE Put"))
fig.add_trace(go.Histogram(x=S_TSX_Bootstrapping, histnorm="probability", nbinsx=100), row=1,col=1)
fig.add_trace(go.Histogram(x=S_BCE_Bootstrapping, histnorm="probability", nbinsx=100), row=1,col=2)
fig.add_trace(go.Histogram(x=Asian_call_Option_Bootstrapping, histnorm="probability", nbinsx=100), row=2,col=1)
fig.add_trace(go.Histogram(x=TSX_call_option_Bootstrapping, histnorm="probability", nbinsx=100), row=2,col=2)
fig.add_trace(go.Histogram(x=BCE_digital_put_Bootstrapping, histnorm="probability", nbinsx=100), row=3,col=1)
fig.add_trace(go.Histogram(x=BCE_Put_Bootstrapping, histnorm="probability", nbinsx=100), row=3,col=2)

fig.update_layout(showlegend=False, title_text="Bootstrapping simulation")

In [0]:
fig = make_subplots(rows=2, cols=2, subplot_titles=("TSX Position", "BCE Position", "Portfolio value", "Loss"))
fig.add_trace(go.Histogram(x=TSX_positions_Bootstrapping, histnorm="probability", nbinsx=100), row=1,col=1)
fig.add_trace(go.Histogram(x=BCE_positions_Bootstrapping, histnorm="probability", nbinsx=100), row=1,col=2)
fig.add_trace(go.Histogram(x=portfolio_value_Bootstrapping, histnorm="probability", nbinsx=100), row=2,col=1)
fig.add_trace(go.Histogram(x=loss_Bootstrapping, histnorm="probability", nbinsx=100), row=2,col=2)
fig.update_layout(showlegend=False)

In [0]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=loss,histnorm="probability", nbinsx=100, name="Loss", xbins=dict( # bins used for histogram
        start=-150000,
        end=50000,
        size=1000
    ),))
fig.add_trace(go.Scatter(x=[loss_95_percentile_Bootstrapping,loss_95_percentile_Bootstrapping], y=[0.00005,0.07], name="95% Loss"))
fig.add_trace(go.Scatter(x=[loss_99_percentile_Bootstrapping,loss_99_percentile_Bootstrapping], y=[0.00005,0.07], name="99% Loss"))
fig.update_layout(title_text="Portfolio Loss")